Делал на скорую руку, поэтому много чего не попробовал:
1) Сделать полиномы, а потом PCA, отобрав лучшие фичи
2) Трансформация фичей
3) Более детальный подбор гиперпараметров для каждой модели
4) Нет графического анализа моделей
 
По сути просто построил бустинг, деревья, svc и knn, слегка подобрал параметры для них и просто стакнул, считается достаточно долго, поэтому лучше применять часть датасета. Результатом не доволен, потом надо доделать

In [3]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

from sklearn.model_selection import train_test_split, GridSearchCV, \
                                    ShuffleSplit, KFold

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

from sklearn.preprocessing import MinMaxScaler, StandardScaler, \
                                  RobustScaler, PolynomialFeatures, \
                                  OrdinalEncoder, LabelEncoder, \
                                  OneHotEncoder, TargetEncoder, \
                                  QuantileTransformer, PowerTransformer, \
                                  KBinsDiscretizer

from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, RANSACRegressor, \
                                 Ridge, Lasso, LinearRegression

from sklearn.metrics import recall_score, precision_score, \
                            f1_score, ConfusionMatrixDisplay, \
                            confusion_matrix, roc_auc_score, \
                            RocCurveDisplay, PrecisionRecallDisplay, \
                            roc_curve, precision_recall_curve, \
                            PrecisionRecallDisplay, make_scorer, \
                            mean_squared_error

from sklearn.compose import ColumnTransformer, make_column_selector,\
                            make_column_transformer, TransformedTargetRegressor
# import category_encoders as ce
from sklearn.metrics import fbeta_score
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime
import matplotlib.pyplot as plt
from collections import OrderedDict
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_validate
from sklearn.ensemble import VotingClassifier
import pickle

In [5]:
data_raw = pd.read_csv('train.csv')
data_raw.head(10)

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,0.0,1.0,23.0,1.0,0.0,1.0,1.0,1.0,0.0,...,1.0,0.0,4.0,5.0,2.0,0.0,0.0,6.0,6.0,3.0
1,1.0,1.0,1.0,35.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,4.0,15.0,15.0,0.0,0.0,10.0,4.0,5.0
2,1.0,0.0,1.0,30.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,3.0,30.0,30.0,0.0,0.0,4.0,5.0,1.0
3,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,1.0,8.0,6.0,7.0
4,0.0,0.0,1.0,36.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,3.0,0.0,2.0,0.0,0.0,5.0,4.0,5.0
5,1.0,1.0,1.0,33.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,7.0,4.0,6.0
6,1.0,0.0,1.0,21.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,2.0,0.0,21.0,0.0,1.0,11.0,6.0,5.0
7,0.0,1.0,1.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,3.0,0.0,2.0,0.0,0.0,1.0,3.0,5.0
8,0.0,0.0,1.0,27.0,0.0,1.0,1.0,1.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,13.0,4.0,8.0
9,1.0,0.0,1.0,31.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,3.0,0.0,7.0,0.0,0.0,13.0,6.0,8.0


In [6]:
display(data_raw.info())
display(data_raw.describe())
display(data_raw.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47363 entries, 0 to 47362
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   HighBP                47363 non-null  float64
 1   HighChol              47363 non-null  float64
 2   CholCheck             47363 non-null  float64
 3   BMI                   47363 non-null  float64
 4   Smoker                47363 non-null  float64
 5   Stroke                47363 non-null  float64
 6   HeartDiseaseorAttack  47363 non-null  float64
 7   PhysActivity          47363 non-null  float64
 8   Fruits                47363 non-null  float64
 9   Veggies               47363 non-null  float64
 10  HvyAlcoholConsump     47363 non-null  float64
 11  AnyHealthcare         47363 non-null  float64
 12  NoDocbcCost           47363 non-null  float64
 13  GenHlth               47363 non-null  float64
 14  MentHlth              47363 non-null  float64
 15  PhysHlth           

None

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
count,47363.000000,47363.000000,47363.000000,47363.000000,47363.000000,47363.000000,47363.000000,47363.000000,47363.000000,47363.000000,...,47363.000000,47363.000000,47363.000000,47363.000000,47363.000000,47363.000000,47363.000000,47363.000000,47363.000000,47363.000000
mean,0.563879,0.523468,0.975741,29.862551,0.475118,0.062834,0.148766,0.701940,0.610814,0.788844,...,0.954585,0.094145,2.835272,3.732829,5.788105,0.252349,0.457213,8.587294,4.923485,5.703292
std,0.495908,0.499454,0.153855,7.144498,0.499386,0.242666,0.355861,0.457411,0.487571,0.408133,...,0.208215,0.292034,1.112528,8.145554,10.044184,0.434365,0.498171,2.847747,1.027849,2.174629
min,0.000000,0.000000,0.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,1.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,7.000000,4.000000,4.000000
50%,1.000000,1.000000,1.000000,29.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,...,1.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,9.000000,5.000000,6.000000
75%,1.000000,1.000000,1.000000,33.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,...,1.000000,0.000000,4.000000,2.000000,5.000000,1.000000,1.000000,11.000000,6.000000,8.000000
max,1.000000,1.000000,1.000000,98.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,5.000000,30.000000,30.000000,1.000000,1.000000,13.000000,6.000000,8.000000


HighBP                  0
HighChol                0
CholCheck               0
BMI                     0
Smoker                  0
Stroke                  0
HeartDiseaseorAttack    0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
AnyHealthcare           0
NoDocbcCost             0
GenHlth                 0
MentHlth                0
PhysHlth                0
DiffWalk                0
Sex                     0
Age                     0
Education               0
Income                  0
dtype: int64

In [7]:
display(data_raw.shape)

(47363, 21)

In [8]:
train_y = pd.read_csv('train_Y.csv')

In [9]:
display(train_y.shape)

(47363, 1)

In [10]:
data_1 = data_raw.drop_duplicates(keep='first')

In [11]:
display(data_1.shape)

(46384, 21)

In [12]:
data_all = pd.concat([data_raw, train_y], axis=1)
display(data_all.head(10))

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes_binary
0,0.0,0.0,1.0,23.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.0,4.0,5.0,2.0,0.0,0.0,6.0,6.0,3.0,1.0
1,1.0,1.0,1.0,35.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,15.0,15.0,0.0,0.0,10.0,4.0,5.0,1.0
2,1.0,0.0,1.0,30.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,3.0,30.0,30.0,0.0,0.0,4.0,5.0,1.0,1.0
3,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,2.0,0.0,0.0,0.0,1.0,8.0,6.0,7.0,1.0
4,0.0,0.0,1.0,36.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,3.0,0.0,2.0,0.0,0.0,5.0,4.0,5.0,1.0
5,1.0,1.0,1.0,33.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,3.0,0.0,0.0,0.0,1.0,7.0,4.0,6.0,1.0
6,1.0,0.0,1.0,21.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,2.0,0.0,21.0,0.0,1.0,11.0,6.0,5.0,1.0
7,0.0,1.0,1.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,2.0,0.0,0.0,1.0,3.0,5.0,1.0
8,0.0,0.0,1.0,27.0,0.0,1.0,1.0,1.0,0.0,1.0,...,0.0,3.0,0.0,0.0,0.0,1.0,13.0,4.0,8.0,1.0
9,1.0,0.0,1.0,31.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,3.0,0.0,7.0,0.0,0.0,13.0,6.0,8.0,1.0


In [13]:
display(data_all.shape)

(47363, 22)

In [14]:
data_all.columns

Index(['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke',
       'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income',
       'Diabetes_binary'],
      dtype='object')

In [15]:
data = data_all.drop_duplicates(keep='first')

In [16]:
display(data.shape)

(46578, 22)

In [17]:
data_2 = data_all.drop_duplicates(keep='first', subset=['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke',
       'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income'])
display(data_2.shape)

(46384, 22)

In [18]:
data_2['Diabetes_binary'].value_counts()

Diabetes_binary
1.0    23468
0.0    22916
Name: count, dtype: int64

In [19]:
X_train, X_test, y_train, y_test = train_test_split(data_2.drop(columns='Diabetes_binary'), data_2['Diabetes_binary'], random_state=13)

In [20]:
beta = 2 ** (1 / 2)

In [21]:
# numeric_columns = data_2.select_dtypes(include=['float64', 'uint64', 'int64'])
# correlation_matrix = numeric_columns.corr(method = 'pearson')
# mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
# plt.figure(figsize=(9, 7))
# sns.heatmap(correlation_matrix, mask=mask, annot=True, fmt=".2f", cmap='coolwarm', linewidths=0.5, square=True)
# plt.title('Корреляционная матрица для числовых данных', fontsize=16)
# plt.show()

In [22]:
model_pipeline_f = Pipeline(steps=[
    ('preprocessing', StandardScaler()), #забиваем свой препроцессор, если надо, можно написать в feature engineering universal_preprocessor
    ('model', RandomForestClassifier(n_estimators=2000, random_state=13,
                               oob_score = True, n_jobs=-1, criterion='entropy'))]) #забиваем свою модель, если надо
display(model_pipeline_f)

Pipeline(steps=[('preprocessing', StandardScaler()),
                ('model',
                 RandomForestClassifier(criterion='entropy', n_estimators=2000,
                                        n_jobs=-1, oob_score=True,
                                        random_state=13))])

In [23]:
# X_train, X_test, y_train, y_test = train_test_split(data_2.drop(), cancer.target, random_state=0)
start_time = datetime.now()
# forest = RandomForestClassifier(n_estimators=1000, random_state=13,
#                                oob_score = True, n_jobs=-1) 
model_pipeline_f.fit(X_train, y_train)
print('Правильность на обучающем наборе: {:.3f}'.format(model_pipeline_f.score(X_train, y_train)))
print('Правильность на OOB: {:.3f}'.format(model_pipeline_f['model'].oob_score_))
y_train_pred = model_pipeline_f.predict(X_train)
y_test_pred = model_pipeline_f.predict(X_test)
# print('Правильность на тестовом наборе: {:.3f}'.format(forest.score(X_test, y_test)))
# do your work here
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Правильность на обучающем наборе: 1.000
Правильность на OOB: 0.740
Duration: 0:00:12.376427


In [24]:
train_fbeta = fbeta_score(y_train, y_train_pred, beta=beta)
test_fbeta = fbeta_score(y_test, y_test_pred, beta=beta)
display(f'Правильность на обучающем наборе для леса:{train_fbeta}')
display(f'Правильность на тестовом наборе для леса:{test_fbeta}')

'Правильность на обучающем наборе для леса:1.0'

'Правильность на тестовом наборе для леса:0.7642236432202001'

In [25]:
forest = model_pipeline_f.fit(data_2.drop(columns='Diabetes_binary'), data_2['Diabetes_binary'])
# with open('forest_model.pkl', 'wb') as f:
#      pickle.dump(forest, f)

In [26]:
pipeline_gbrt = Pipeline(steps=[
    ('preprocessing', StandardScaler()), #забиваем свой препроцессор, если надо, можно написать в feature engineering universal_preprocessor
    ('model', GradientBoostingClassifier(random_state=13, loss='exponential', max_depth=4, n_estimators=4000, learning_rate=0.01, subsample=0.75))]) #забиваем свою модель, если надо
display(pipeline_gbrt)

Pipeline(steps=[('preprocessing', StandardScaler()),
                ('model',
                 GradientBoostingClassifier(learning_rate=0.01,
                                            loss='exponential', max_depth=4,
                                            n_estimators=4000, random_state=13,
                                            subsample=0.75))])

In [27]:
# gbrt = GradientBoostingClassifier(random_state=13, max_depth=5, n_estimators=300)
pipeline_gbrt.fit(X_train, y_train)
print('Правильность на обучающем наборе: {:.3f}'.format(pipeline_gbrt.score(X_train, y_train)))
print('Правильность на тестовом наборе: {:.3f}'.format(pipeline_gbrt.score(X_test, y_test)))
y_train_pred = pipeline_gbrt.predict(X_train)
y_test_pred = pipeline_gbrt.predict(X_test)

Правильность на обучающем наборе: 0.771
Правильность на тестовом наборе: 0.749


In [28]:
train_fbeta = fbeta_score(y_train, y_train_pred, beta=beta) 
test_fbeta = fbeta_score(y_test, y_test_pred, beta=beta)
display(f'Правильность на обучающем наборе для бустинга:{train_fbeta}')
display(f'Правильность на тестовом наборе для бустинга:{test_fbeta}')

'Правильность на обучающем наборе для бустинга:0.7948982601002654'

'Правильность на тестовом наборе для бустинга:0.7758507853403143'

In [29]:
gbrt = pipeline_gbrt.fit(data_2.drop(columns='Diabetes_binary'), data_2['Diabetes_binary'])
# with open('gbrt_model.pkl', 'wb') as f:
#      pickle.dump(gbrt, f)

In [30]:
pipeline_KNN = Pipeline(steps=[
    ('preprocessing', StandardScaler()), #забиваем свой препроцессор, если надо, можно написать в feature engineering universal_preprocessor
    ('model', KNeighborsClassifier(n_neighbors=24, p=4))]) #забиваем свою модель, если надо

In [31]:
pipeline_svc = Pipeline(steps=[
    ('preprocessing', StandardScaler()), #забиваем свой препроцессор, если надо, можно написать в feature engineering universal_preprocessor
    ('model', SVC(kernel='rbf', gamma=8, C=1))]) #забиваем свою модель, если надо
display(pipeline_svc)

Pipeline(steps=[('preprocessing', StandardScaler()),
                ('model', SVC(C=1, gamma=8))])

In [32]:
start_time = datetime.now()
clf_v1 = RandomForestClassifier(n_estimators=1000, random_state=13,
                               oob_score = True, n_jobs=-1, criterion='entropy')
clf_v2 = GradientBoostingClassifier(random_state=13, loss='exponential', max_depth=4, n_estimators=4000, learning_rate=0.01, subsample=0.75)
clf_v3 = pipeline_svc
clf_v4 = pipeline_KNN


e_s_clf = StackingClassifier(
        estimators = [
            ('RF', clf_v1), 
            ('GB', clf_v2), 
            ('SVC', clf_v3),
            ('KNN', clf_v4)],
        final_estimator = LogisticRegression(max_iter=100),
        stack_method = 'predict'
)

e_s_clf.fit(X_train.iloc[:7000],y_train[:7000])
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:01:33.164194


In [33]:
y_train_pred = e_s_clf.predict(X_train)
y_test_pred = e_s_clf.predict(X_test)

In [34]:
train_fbeta = fbeta_score(y_train, y_train_pred, beta=beta) 
test_fbeta = fbeta_score(y_test, y_test_pred, beta=beta)
display(f'Правильность на обучающем наборе для бустинга:{train_fbeta}')
display(f'Правильность на тестовом наборе для бустинга:{test_fbeta}')

'Правильность на обучающем наборе для бустинга:0.7895421386967258'

'Правильность на тестовом наборе для бустинга:0.7587406793703397'

In [36]:
# Stack = e_s_clf.fit(data_2.drop(columns='Diabetes_binary'), data_2['Diabetes_binary'])
# with open('stack_model.pkl', 'wb') as f:
#      pickle.dump(Stack, f)

In [ ]:
# start_time = datetime.now()
# clf_vs1 = RandomForestClassifier(n_estimators=2000, random_state=13,
#                                oob_score = True, n_jobs=-1, criterion='entropy')
# clf_vs2 = GradientBoostingClassifier(random_state=13, loss='exponential', max_depth=4, n_estimators=4000, learning_rate=0.027)
# clf_vs3 = SVC(kernel='rbf', probability=True)

# e_vs_clf = VotingClassifier(estimators=[('RF', clf_vs1), ('GBC', clf_vs2), ('svc', clf_vs3)],
#                         voting='soft', weights=[2, 2, 1])
# e_vs_clf.fit(X_train, y_train)
# end_time = datetime.now()
# print('Duration: {}'.format(end_time - start_time))

In [ ]:
# y_train_pred = e_s_clf.predict(X_train)
# y_test_pred = e_s_clf.predict(X_test)

In [ ]:
# train_fbeta = fbeta_score(y_train, y_train_pred, beta=beta) 
# test_fbeta = fbeta_score(y_test, y_test_pred, beta=beta)
# display(f'Правильность на обучающем наборе для бустинга:{train_fbeta}')
# display(f'Правильность на тестовом наборе для бустинга:{test_fbeta}')

In [ ]:
# pipeline_svc.fit(X_train, y_train)
# print('Правильность на обучающем наборе: {:.3f}'.format(pipeline_svc.score(X_train, y_train)))
# print('Правильность на тестовом наборе: {:.3f}'.format(pipeline_svc.score(X_test, y_test)))
# y_train_pred = pipeline_svc.predict(X_train)
# y_test_pred = pipeline_svc.predict(X_test)

In [ ]:
# train_fbeta = fbeta_score(y_train, y_train_pred, beta=beta) 
# test_fbeta = fbeta_score(y_test, y_test_pred, beta=beta)
# display(f'Правильность на обучающем наборе для бустинга:{train_fbeta}')
# display(f'Правильность на тестовом наборе для бустинга:{test_fbeta}')

In [ ]:
# pipeline_KNN = Pipeline(steps=[
#     ('preprocessing', StandardScaler()), #забиваем свой препроцессор, если надо, можно написать в feature engineering universal_preprocessor
#     ('model', KNeighborsClassifier(n_neighbors=5))]) #забиваем свою модель, если надо
# display(pipeline_svc)

In [ ]:
# pipeline_svc.fit(X_train, y_train)
# print('Правильность на обучающем наборе: {:.3f}'.format(pipeline_svc.score(X_train, y_train)))
# print('Правильность на тестовом наборе: {:.3f}'.format(pipeline_svc.score(X_test, y_test)))
# y_train_pred = pipeline_svc.predict(X_train)
# y_test_pred = pipeline_svc.predict(X_test)

In [ ]:
# train_fbeta = fbeta_score(y_train, y_train_pred, beta=beta)
# test_fbeta = fbeta_score(y_test, y_test_pred, beta=beta)
# display(f'Правильность на обучающем наборе для бустинга:{train_fbeta}')
# display(f'Правильность на тестовом наборе для бустинга:{test_fbeta}')

In [ ]:
# cv = ShuffleSplit(n_splits=4, random_state=13)
# pipe_kneighbors = Pipeline([('preprocessing', StandardScaler()), ('classifier', SVC())])

# C=[0.01, 0.1, 1, 10, 100, 1000]
# # p=[4, 6]
# # weights=['uniform','distance']

# param_grid =[
#     {'preprocessing': [MinMaxScaler()],
#      'classifier': [SVC()],
#      'classifier__C': C,
#      }
# ]

# grid = GridSearchCV(pipe_kneighbors, param_grid, cv=cv, return_train_score=True, n_jobs=-1)
# grid.fit(X_train.iloc[:5000], y_train.iloc[:5000].values.ravel())

# print("----------------- Обучили и тестировали -------------------")
# print("Наилучшие параметры:\n{}\n".format(grid.best_params_))
# print("Средняя правильность для наилучшей модели кроссвалидации на \
#                 валидационных тестовых наборах: {:.6f}\n".format(grid.best_score_)) 
# gridresults = pd.DataFrame(grid.cv_results_)
# display(gridresults.sort_values(["rank_test_score"]).T)

In [ ]:
# matrix = confusion_matrix(target_test, grid.predict(features_test))
# ConfusionMatrixDisplay(matrix).plot()

In [ ]:
# pipe_1 = make_pipeline(MinMaxScaler(), KNeighborsClassifier(n_jobs=-1, n_neighbors=30, p=4, weights='distance'))
# pipe_1.fit(features_train,target_train)
# print("Правильность модели на тестовом наборе: {:.6f}\n".format(precision_score(target_test, pipe_1.predict(features_test))))
# RocCurveDisplay.from_estimator(pipe_1, features_test, target_test, name = 'pipe');

In [ ]:
# train_fbeta = fbeta_score(y_train, y_train_pred, beta=beta)
# test_fbeta = fbeta_score(y_test, y_test_pred, beta=beta)
# display(f'Правильность на обучающем наборе для бустинга:{train_fbeta}')
# display(f'Правильность на тестовом наборе для бустинга:{test_fbeta}')